# BIOMASS telemetry

In [1]:
import os
from datetime import datetime, timedelta
from typing import BinaryIO, Literal, Optional, Union

import bpack
import pandas as pd
from tqdm import tqdm

from biotm import tm

In [2]:
%load_ext watermark
%watermark -i -v -p bpack,biotm

Python implementation: CPython
Python version       : 3.9.4
IPython version      : 7.22.0

bpack: 0.7.1
biotm: 0.2



In [3]:
F_USO = 37.826087e6

def stream_to_dataframe(file: Union[str, os.PathLike],
                        isp_type: Literal['generic', 'sar', 'platform'] = 'generic',
                        trailing_spare_bytes: Optional[int] = None) -> pd.DataFrame:
    
    with open(file, 'rb') as fh, tqdm(total=os.stat(file).st_size, desc=file, unit=' bytes') as pbar:

        stream = tm.ISPStream(fh)
        records = []

        for n, isp in enumerate(stream.iterpackets(isp_type)):
            
            record = {
                'total_size': tm.PACKET_HEADER_SIZE + isp.packet_header.pdfl + 1,
                'timestamp': pd.Timestamp('2000-01-01')
            }
            record.update(bpack.asdict(isp.packet_header))
            record.update(bpack.asdict(isp.data_field_header))
            seconds = isp.data_field_header.ct
            
            if isp_type == 'sar':
         
                sar_header = isp.source_data.sar_header()
                seconds += (isp.data_field_header.ft * 2 ** 8 + sar_header.fte) * 4 / F_USO
                record['fte'] = sar_header.fte
                record.update(bpack.asdict(sar_header.obt_stat))
                record.update(synch=sar_header.synch, spct=sar_header.spct, prict=sar_header.prict)
                record.update(bpack.asdict(sar_header.data_take_information_service))
                record.update(
                    {k: v for k, v in bpack.asdict(sar_header.real_time_parameter_service).items()
                     if k not in ('sspa', 'rcm', 'sidx', 'spares')}
                )
                record['user_data_size'] = len(isp.source_data.user_data())
            
            elif isp_type == 'generic':
                
                seconds += isp.data_field_header.ft * 2 ** -16
                
            elif isp_type == 'platform':
            
                seconds += isp.data_field_header.ft * 2 ** -16   
                plt_anc_data = isp.source_data.platform_ancillary_data(trailing_spare_bytes)
                record.update(
                    aocs_version=plt_anc_data.aocs_version,
                    drift_flag=plt_anc_data.drift_flag,
                    nav_ut=plt_anc_data.nav_ut.day + plt_anc_data.nav_ut.dayf,
                    nav_dt=plt_anc_data.nav_dt,
                    nav_valid=plt_anc_data.nav_valid,
                    nav_gnss_pv_date=plt_anc_data.nav_gnss_pv_date,
                    nav_gnss_pvt_val=plt_anc_data.nav_gnss_pvt_val
                )
                
            else:
                
                raise ValueError(f"unknown ISP type: '{isp_type}'")

            record['timestamp'] += pd.to_timedelta(seconds, 'sec')
            pbar.update(record['total_size'])
            records.append(record)

    df = pd.DataFrame(records)
    df.insert(1, 'offset', df['total_size'].cumsum() - df['total_size'])

    return df


Decode SAR ISP as generic ISP:

In [4]:
df = stream_to_dataframe('BIO_OPER_RAW_XB_SP__2510NNNN_20250101T060251.294.isp')

BIO_OPER_RAW_XB_SP__2510NNNN_20250101T060251.294.isp: 100%|██████████| 323814400/323814400 [00:13<00:00, 23287915.89 bytes/s]


In [5]:
df.head()

,total_size,offset,timestamp,pktvers,type,dfh,pid,pcat,gf,ssc,pdfl,pusvers,serv,subserv,dest,ct,ft
0,1792,0,2025-01-01 06:03:28.165847778,0,0,1,25,10,3,0,1785,1,211,1,0,789026608,10869
1,1792,1792,2025-01-01 06:03:28.166030884,0,0,1,25,10,3,1,1785,1,211,1,0,789026608,10881
2,1792,3584,2025-01-01 06:03:28.166213989,0,0,1,25,10,3,2,1785,1,211,1,0,789026608,10893
3,1792,5376,2025-01-01 06:03:28.166397095,0,0,1,25,10,3,3,1785,1,211,1,0,789026608,10905
4,1792,7168,2025-01-01 06:03:28.166580200,0,0,1,25,10,3,4,1785,1,211,1,0,789026608,10917


Decode SAR ISP as SAR ISP:

In [6]:
df = stream_to_dataframe('BIO_OPER_RAW_XB_SP__2510NNNN_20250101T060251.294.isp', isp_type='sar')

BIO_OPER_RAW_XB_SP__2510NNNN_20250101T060251.294.isp: 100%|██████████| 323814400/323814400 [00:42<00:00, 7561683.93 bytes/s]


In [7]:
df.head()

,total_size,offset,timestamp,pktvers,type,dfh,pid,pcat,gf,ssc,...,swp,swl,bitsel,baqmode,styp,txpidx,rank,tcomp,txpl,user_data_size
0,1792,0,2025-01-01 06:03:28.294261098,0,0,1,25,10,3,0,...,3382,6720,224,1,1,0,15,False,726,1698
1,1792,1792,2025-01-01 06:03:28.294586539,0,0,1,25,10,3,1,...,3382,6720,224,1,0,0,15,False,726,1698
2,1792,3584,2025-01-01 06:03:28.294911861,0,0,1,25,10,3,2,...,3382,6720,224,1,1,0,15,False,726,1698
3,1792,5376,2025-01-01 06:03:28.295237184,0,0,1,25,10,3,3,...,3382,6720,224,1,0,0,15,False,726,1698
4,1792,7168,2025-01-01 06:03:28.295562744,0,0,1,25,10,3,4,...,3382,6720,224,1,1,0,15,False,726,1698


Decode platform ancillary ISP as generic ISP:

In [8]:
df = stream_to_dataframe('BIO_OPER_RAW_XB_SP__2210NNNN_20250101T060241.294.isp')

BIO_OPER_RAW_XB_SP__2210NNNN_20250101T060241.294.isp: 100%|██████████| 23868/23868 [00:00<00:00, 3231011.10 bytes/s]


In [9]:
df.head()

,total_size,offset,timestamp,pktvers,type,dfh,pid,pcat,gf,ssc,pdfl,pusvers,serv,subserv,dest,ct,ft
0,306,0,2025-01-01 06:03:18.460418701,0,0,1,22,10,3,0,299,1,210,2,0,789026598,30174
1,306,306,2025-01-01 06:03:19.460418701,0,0,1,22,10,3,1,299,1,210,2,0,789026599,30174
2,306,612,2025-01-01 06:03:20.460418701,0,0,1,22,10,3,2,299,1,210,2,0,789026600,30174
3,306,918,2025-01-01 06:03:21.460418701,0,0,1,22,10,3,3,299,1,210,2,0,789026601,30174
4,306,1224,2025-01-01 06:03:22.460418701,0,0,1,22,10,3,4,299,1,210,2,0,789026602,30174


Decode platform ancillary ISP as platform ancillary ISP:

In [10]:
df = stream_to_dataframe('BIO_OPER_RAW_XB_SP__2210NNNN_20250101T060241.294.isp', isp_type='platform', trailing_spare_bytes=4)

BIO_OPER_RAW_XB_SP__2210NNNN_20250101T060241.294.isp: 100%|██████████| 23868/23868 [00:00<00:00, 2594051.82 bytes/s]


In [11]:
df.head()

,total_size,offset,timestamp,pktvers,type,dfh,pid,pcat,gf,ssc,...,dest,ct,ft,aocs_version,drift_flag,nav_ut,nav_dt,nav_valid,nav_gnss_pv_date,nav_gnss_pvt_val
0,306,0,2025-01-01 06:03:18.460418701,0,0,1,22,10,3,0,...,0,789026598,30174,0,0,60676.251867,0.0,0,0.0,0
1,306,306,2025-01-01 06:03:19.460418701,0,0,1,22,10,3,1,...,0,789026599,30174,0,0,60676.251878,0.0,0,0.0,0
2,306,612,2025-01-01 06:03:20.460418701,0,0,1,22,10,3,2,...,0,789026600,30174,0,0,60676.251890,0.0,0,0.0,0
3,306,918,2025-01-01 06:03:21.460418701,0,0,1,22,10,3,3,...,0,789026601,30174,0,0,60676.251902,0.0,0,0.0,0
4,306,1224,2025-01-01 06:03:22.460418701,0,0,1,22,10,3,4,...,0,789026602,30174,0,0,60676.251913,0.0,0,0.0,0
